In [1]:
import pandas as pd
import numpy as np
import jieba

'''
選擇讀取的正負評資料，原始資料正評31778則，負評5679則
'''

df1 = pd.read_csv('good_31778.csv', encoding='utf-8')
#df1 = pd.read_csv('good_5679.csv', encoding='utf-8')
df2 = pd.read_csv('bad_5679.csv', encoding='utf-8')

col_name1=df1.columns.tolist()
col_name2=df2.columns.tolist()
col_name1.insert(2,'score')
col_name2.insert(2,'score')
df1['score']=1
df2['score']=2
df_total=pd.concat([df1, df2])

#新增cut_comment欄位為後續使用

df_total['cut_comment']=" "
jieba.set_dictionary('jieba_data/dict.txt.big')

In [2]:
df_total

,comment,score,cut_comment
0,年瑞凡菲尼克斯猝逝他與強納森普萊斯茱蒂戴維斯共同演出的黑血在完成百分之九十左右的拍攝進度後就...,1,
1,雷文防雷資訊頁雷文主文分隔線新聞報導這部電影說是「超越前座」個人覺得過譽了前面還可以尤其是鋪...,1,
2,上一次看電影已經是四個月以前了終於趁著桃園新光影城新開幕剛好期待許久的打噴嚏今天全天場次怎麼...,1,
3,月日上架了一部漫畫改編的少年奇幻劇修女戰士緊接著下個週末月日又上架了一部漫畫改編的少年動作片...,1,
4,抬尬后小弟身為一個忠實原作小說粉今天一早早場就去看了其實應該是普雷的但是為了騙各位進來所以打...,1,
...,...,...,...
5674,雷文防雷資訊頁影片名稱無雙觀影時間月觀影地點戲院名稱秀泰觀影方式自行購票§其他防雷說明非必要...,2,
5675,雷文防雷資訊頁影片名稱來了一個男孩之後觀影時間觀影地點金馬影展觀影方式自行購票§其他防雷說明...,2,
5676,雷文主文分隔線網誌版身為恐怖片的電影粉就算在電影版看到別人的影評列為級恐怖片之輩還是興沖沖地...,2,
5677,雷文防雷資訊頁影片名稱誰先愛上他的觀影時間觀影地點台南真善美觀影方式自行購票夾銗防雷說明非必...,2,


In [3]:
import re
number = 0

'''
未縮減資料或縮減資料row數不同，在此先將comment由jieba斷字斷詞後存入cut_comment欄位
斷字斷詞也可以放在向量化時再進行，但同時執行可能占用太多記憶體，所以分成二步
'''

for i in range(37457):   # 31778:5679
#for i in range(11358):   # 5679:5679

    comment = df_total.iloc[number, 0]
    comment = re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]", "",str(comment))
    df_total.iloc[number, 2] = ' '.join(jieba.cut(comment, cut_all=False, HMM=True))
    number += 1

Building prefix dict from C:\Users\Big data\PycharmProjects\train_good_bad\jieba_data\dict.txt.big ...
Loading model from cache C:\Users\BIGDAT~1\AppData\Local\Temp\jieba.u50ec2580c423b29812c80c9b62425765.cache
Loading model cost 0.933 seconds.
Prefix dict has been built successfully.


In [6]:
#31778:5679或5679:5679   未特殊處理使用此段
from sklearn.feature_extraction.text import CountVectorizer 
X = df_total.cut_comment
y = df_total.score
bow_transformer = CountVectorizer().fit(X.values.astype('U'))
X = bow_transformer.transform(X.values.astype('U'))

#分割訓練資料及測試資料

from sklearn.model_selection import train_test_split,cross_val_score
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=99)

from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=0.01)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)

from sklearn.metrics import confusion_matrix, classification_report
#根據預測值和真實值計算相關指標
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           1       0.14      0.13      0.14      1733
           2       0.15      0.16      0.15      1675

    accuracy                           0.14      3408
   macro avg       0.14      0.14      0.14      3408
weighted avg       0.14      0.14      0.14      3408



In [4]:
#31778:5679使用SMOTE演算法進行過採樣
from sklearn.feature_extraction.text import CountVectorizer 
from imblearn.over_sampling import SMOTE
X = df_total.cut_comment
y = df_total.score
oversampler=SMOTE(random_state=0)
bow_transformer = CountVectorizer().fit(X.values.astype('U'))
X_vec = bow_transformer.transform(X.values.astype('U'))
X_resampled, y_resampled = oversampler.fit_sample(X_vec, y)

#分割訓練資料及測試資料

from sklearn.model_selection import train_test_split,cross_val_score
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=99)

from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=0.01)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)


from sklearn.metrics import confusion_matrix, classification_report
#根據預測值和真實值計算相關指標
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           1       0.86      0.85      0.86      9557
           2       0.85      0.86      0.86      9510

    accuracy                           0.86     19067
   macro avg       0.86      0.86      0.86     19067
weighted avg       0.86      0.86      0.86     19067



In [9]:
#儲存模型以供後續使用

import joblib
joblib.dump(clf, "model_31778_5679_SMOTE.m")
joblib.dump(bow_transformer, "trans_31778_5679_SMOTE.m")

['trans_31778_5679_SMOTE.m']